In [147]:
import requests
from requests_html import HTML
import pandas as pd
import os


In [148]:
def url_to_file(url,filename='data.html'):
    r=requests.get(url)
    if r.status_code == 200:
        html_text=r.text
        f=open(filename,'w')
        f.write(html_text)
        return html_text
    return "false"

In [149]:
def parse_and_save(url,year):
    BASR_DIR=os.path.dirname('__file__')
    url=f'https://www.boxofficemojo.com/year/world/{year}/'

    html_text=url_to_file(url)               #extract html text from url
    if html_text=='false':
        print('enter valid year')
    else:
        r_html=HTML(html=html_text)              #create r_html object to extract text from tags
        table_class='.imdb-scroll-table'         # . selector to find table with this class
        r_table=r_html.find(table_class)         # list containing single table      

        if len(r_table)==1:
            table_data=[]                            #list to store rows of table  
            parsed_table=r_table[0]            
            rows=parsed_table.find('tr')             #find rows with tr attribute
            header=rows[0]                           #html text of coloumn names
            col_name=[]                              #list to store col name
            header_name=header.find('th')            #finding col name
            for i in header_name:
                col_name.append(i.text) 
            for x in rows[1:]:                       #perform a for loop for each row
                cols=x.find('td')                    #finding elements of each row
                row_data=[]
                for i,x in enumerate(cols):
                    row_data.append(x.text)
                table_data.append(row_data)
            df=pd.DataFrame(table_data,columns=col_name)
            path=os.path.join(BASR_DIR,'data')
            os.makedirs(path,exist_ok=True)
            file_path=os.path.join('data',f'{year}_box_office_collection.csv')
            df.to_csv(file_path,index=False)

In [150]:
if __name__=="__main__":
    from_yr=input('enter the starting year:')
    to_yr=input('enter the final year:')
    if (from_yr>to_yr):
        print('invalid input')
    else:
        for x in range(int(from_yr),int(to_yr)+1):
            parse_and_save(url,year=str(x))

enter the starting year:2013
enter the final year:2017
